In [40]:
import tensorflow as tf

# Kiểm tra nếu Colab đang sử dụng GPU
print("GPU Available: ", tf.test.is_gpu_available())

# Giả sử bạn đã có mô hình Keras (model)
model = tf.keras.models.load_model('mobilenetv3_fer2013_V3.keras')

# Chuyển đổi mô hình Keras sang TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Lưu mô hình TFLite
with open('mobilenetv3_model.tflite', 'wb') as f:
    f.write(tflite_model)

print("✅ Mô hình đã được chuyển sang định dạng TFLite.")


GPU Available:  True


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 34 variables whereas the saved optimizer has 66 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Saved artifact at '/tmp/tmpp7vhbpf8'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer_3')
Output Type:
  TensorSpec(shape=(None, 7), dtype=tf.float32, name=None)
Captures:
  138363092568400: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138363092569744: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138363092570128: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138363092569936: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138363092568592: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138363092569168: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138363092571856: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138363092572240: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138363092572048: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138363092568208: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1383630925733

In [41]:
import time
import numpy as np
from PIL import Image

# Load mô hình TFLite đã chuyển đổi
interpreter = tf.lite.Interpreter(model_path="mobilenetv3_model.tflite")

# Alocate tensors
interpreter.allocate_tensors()

# Lấy input và output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Resize ảnh input cho phù hợp với kích thước yêu cầu của mô hình
img_size = input_details[0]['shape'][1:3]  # height, width

import os
from PIL import Image
import numpy as np
import time

# Đường dẫn tới thư mục chứa ảnh
image_dir = 'FER2013_Images/PublicTest'  # Đảm bảo đường dẫn chính xác đến thư mục ảnh

image_paths = []
for root, dirs, files in os.walk(image_dir):
    for file in files:
        if file.lower().endswith(('.jpg', '.jpeg', '.png')):  # Kiểm tra các đuôi file ảnh
            image_paths.append(os.path.join(root, file))

inference_times = []

# Thực hiện suy luận trên các ảnh
for path in image_paths:
    img = Image.open(path).resize((img_size[1], img_size[0]))  # resize ảnh về đúng kích thước
    img = np.array(img).astype(np.float32) / 255.0  # chuẩn hóa ảnh

    # Nếu ảnh là grayscale, chuyển thành ảnh RGB
    if len(img.shape) == 2:
        img = np.stack((img,) * 3, axis=-1)

    img = np.expand_dims(img, axis=0)  # thêm batch dimension

    # Thiết lập input tensor
    interpreter.set_tensor(input_details[0]['index'], img)

    # Thực hiện suy luận
    start_time = time.time()
    interpreter.invoke()
    end_time = time.time()

    # Lấy kết quả
    output_data = interpreter.get_tensor(output_details[0]['index'])

    # In thời gian suy luận và kết quả
    inference_time = (end_time - start_time) * 1000  # ms
    inference_times.append(inference_time)
    #print(f"⏱️ Thời gian suy luận: {inference_time:.2f} ms")
    #print(f"📊 Kết quả dự đoán: {output_data}")

# Tính thời gian trung bình suy luận
average_inference_time = np.mean(inference_times)

# In thời gian trung bình suy luận
print(f"⏱️ Thời gian suy luận trung bình: {average_inference_time:.2f} ms")


⏱️ Thời gian suy luận trung bình: 2.64 ms
